In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, date
import json
from pprint import pprint
import requests
import time

URL = "https://www.aclu.org/legislative-attacks-on-lgbtq-rights"

start = time.time()
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
bills = soup.find(id='map').find_all('legtracker')[0][':bills']

# this bill was incorrectly attributed to NC instead of SC
broken = 'https://www.scstatehouse.gov/sess125_2023-2024/bills/585.htm'
def sanitize_item(item):
    rewrite = {'state': {'value': 'SC', 'label': 'South Carolina'}} if 'scstatehouse' in item['link'] else {}
    return {
        **item,
        **rewrite
    }

# for some reason a few of the items in the dataset are from late 2022
def validate_date(item):
    return datetime.strptime(item['status_date'], '%m/%d/%Y').date() > date.fromisoformat('2023-01-01')

contents = [sanitize_item(item) for item in json.loads(bills) if validate_date(item)]

with open('aclu.json', 'w') as outfile:
    json.dump(contents, outfile)

end = time.time()

print(f'Data "aclu.json" refreshed with {len(contents)} items ({(end-start):.2f}s elapsed)')

Data "aclu.json" refreshed with 412 items (0.21s elapsed)
